# Toy study extended

In [ ]:
import alldecays
import matplotlib.pyplot as plt
import numpy as np

# The following import loads a typical setup procedure.
from higgs_data_setup import combi_data_set, _brs


def get_bins(data):
    return np.arange(data.min(), data.max() + 1) - 0.5


def fit_step(minuit_object):
    # minuit_object.throw_nan = True
    minuit_object.print_level = 0
    minuit_object.migrad(ncall=10_000)


fit_mode = "BinomialLeastSquares"
fit = alldecays.Fit(combi_data_set, fit_mode, fit_step=fit_step, has_limits=True)

In [ ]:
rng = np.random.default_rng(seed=5)
toy_values = fit.fill_toys(10_000, store_channel_counts=True)
alldecays.plotting.toy_hists(fit)

In [ ]:
alldecays.plotting.toy_diagnostics_plots(fit)

## Some non-standard graphs

In [ ]:
def get_errors(error_definitions):
    errors = []
    for errordef in error_definitions:

        def fit_step(minuit_object):
            minuit_object.errordef = errordef
            minuit_object.throw_nan = True
            minuit_object.print_level = 0
            minuit_object.migrad(ncall=10_000)

        fit_tmp = alldecays.Fit(
            combi_data_set,
            fit_mode,
            fit_step=fit_step,
            has_limits=True,
            print_brs_sum_not_1=False,
        )
        errors.append(fit_tmp.fit_mode.errors)
    errors = np.concatenate(errors).reshape(-1, len(_brs))
    return errors


error_definitions = [0.5, 1, 2, 4, 8]
x = np.arange(len(error_definitions))
errors = get_errors(error_definitions)
toy_errors = fit.toys.physics.std(axis=0)
fig, ax = plt.subplots(figsize=(6, 8))
for i, br in enumerate(_brs):
    ax.plot(x, errors[:, i], "o--", label=br)
    ax.axhline(toy_errors[i], color=f"C{i}")
ax.set_xticks(x)
ax.set_xticklabels(error_definitions)
ax.set_xlabel("errordef=")
ax.set_ylabel("migrad error")
ax.set_title("Minuit errors\n(horizontal lines: standard deviation of toy fit minima)")
ax.legend(title="decay mode");

In [ ]:
ds_name = "ds:higgs"
box_names = combi_data_set.get_channels()[ds_name].box_names
toy_channel_counts = np.stack([cc[ds_name] for cc in toy_values._channel_counts]).T
for tcc, box_name in zip(toy_channel_counts, box_names):
    fig, ax = plt.subplots()
    ax.set_title(box_name)
    ax.hist(tcc, bins=get_bins(tcc))

In [ ]:
diagnostics = ["fval", "nfcn"]
# diagnostics += ["accurate", "valid"]
fig, axs = plt.subplots(figsize=(4, 4 * len(diagnostics)), nrows=len(diagnostics))
for diag, ax in zip(diagnostics, axs):
    ax.set_title(diag)
    counts = getattr(toy_values, diag)
    if counts.dtype == bool:
        bins = [-0.5, 0.5, 1.5]
    else:
        bins = get_bins(counts)
    ax.hist(counts, bins)